In [1]:
library(SingleCellExperiment)
library(Seurat)
library(tidyverse)
library(SeuratDisk)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
source('../00_preprocessing/preprocessing.R')
source('../01_milo/milo.R')

In [ ]:
# obj_neurons = readRDS('/projects/mludwig/DVC/output/Seurat_objs/integrated/neurons_Seurat_obj_labels.rds')
# obj_neurons
# obj_neurons %>% qs::qsave('dvc_neurons.qs')

In [4]:
obj_neurons = qs::qread('dvc_neurons.qs')
DefaultAssay(obj_neurons) = 'rat_RNA'
obj_neurons[['SCT']] = NULL
obj_neurons[['RNA']] = NULL
obj_neurons[['integrated']] = NULL
obj_neurons[['mouse_RNA']] = NULL
obj_neurons[['macaque_RNA']] = NULL
# obj_neurons[['pca']] = NULL
obj_neurons[['umap']] = NULL
obj_neurons

An object of class Seurat 
32883 features across 218547 samples within 1 assay 
Active assay: rat_RNA (32883 features, 0 variable features)

In [5]:
obj_neurons %>%
`[[` %>%
filter(species == 'rat') %>%
group_by(run) %>%
summarise(n = n())

run,n
<chr>,<int>
rat1,17587
rat2,21171
rat3,20713
rat4,21721
rat5,16370
rat6,24306


In [6]:
rat_cells = obj_neurons %>%
`[[` %>%
filter(species == 'rat') %>%
rownames

In [7]:
length(rat_cells)

[1] 121868

In [8]:
obj = obj_neurons %>%
      subset(cells = rat_cells) %>%
      sample_n_cells_from_obj(20000)
obj

An object of class Seurat 
32883 features across 20000 samples within 1 assay 
Active assay: rat_RNA (32883 features, 0 variable features)

In [9]:
obj = obj %>%
      Seurat::SCTransform(assay='rat_RNA',
                          method="glmGamPoi",
                          verbose=TRUE,
                          variable.features.n = 3000) %>%
      RunPCA(npcs = 100) %>%
      RunUMAP(dims = 1:30)
obj

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 22567 by 20000

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |======================================================================| 100%


Found 97 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 22567 genes



  |======================================================================| 100%


Computing corrected count matrix for 22567 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 5.90277 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is Seurat..SCTransform.rat_RNA; see ?make.names for more details on syntax validity”
PC_ 1 
Positive:  Ubb, Ndufa4, Tmsb4x, Stmn1, Hsp90ab1, Rpl19, Pebp1, Resp18, Atp6v0c, Cox4i1 
	   LOC100364435, Aldoa, LOC687780, Caly, Slc25a4, Stmn3, Calm1, Cox5b, Atp5if1, Fth1 
	   Tpt1, Cox7b, Sst, Cox6c, AABR07000398.1, Rpl23, Cox6a1, Rps8, Ndufb8, Cox8a 
Negative:  Il1rapl2, Lrrtm4, Il1rapl1, Htr2c, LOC304725, AABR07057765.1, AABR07060487.1, Grm8, Hs6st3, Rbfox1 
	   AC134224.1, Gpc6, Kcnip1, Brinp3, Mctp1, Cacna2d3, Lrfn5, Ndst4, Dgkb, Cadps 
	   Gria4, RGD1564053, Cntnap5b, Vwc2l, Cntnap5c, Adgrb3, Cdh12, Thsd7b, Ano4, Fat3 
PC_ 2 
Positive:  Il1rapl2, AABR07033570.1, LOC304725, Grik1, Nefm, Grm8,

An object of class Seurat 
55450 features across 20000 samples within 2 assays 
Active assay: SCT (22567 features, 3000 variable features)
 1 other assay present: rat_RNA
 2 dimensional reductions calculated: pca, umap

In [10]:
obj %>% qs::qsave('dvc_neurons_rat_20kc_3kvf.qs')

In [11]:
obj

An object of class Seurat 
55450 features across 20000 samples within 2 assays 
Active assay: SCT (22567 features, 3000 variable features)
 1 other assay present: rat_RNA
 2 dimensional reductions calculated: pca, umap

In [12]:
SaveH5Seurat(obj, filename = "/scratch/nmq407/dvc_neurons_rat_20kc_3kvf.h5Seurat", overwrite = TRUE)

Creating h5Seurat file for version 3.1.5.9900

Adding counts for rat_RNA

Adding data for rat_RNA

No variable features found for rat_RNA

No feature-level metadata found for rat_RNA

Adding counts for SCT

Adding data for SCT

Adding scale.data for SCT

Adding variable features for SCT

No feature-level metadata found for SCT

Writing out SCTModel.list for SCT

Adding cell embeddings for pca

Adding loadings for pca

No projected loadings for pca

Adding standard deviations for pca

No JackStraw data for pca

Adding cell embeddings for umap

No loadings for umap

No projected loadings for umap

No standard deviations for umap

No JackStraw data for umap



In [21]:
Convert("/scratch/nmq407/dvc_neurons_rat_20kc_3kvf.h5Seurat", 
        dest = "/scratch/nmq407/dvc_neurons_rat_20kc_3kvf.h5Seurat.h5ad", 
        assay='SCT',
       overwrite=TRUE)

Validating h5Seurat file

Adding scale.data from SCT as X

Adding data from SCT as raw

Transfering meta.data to obs

Adding dimensional reduction information for pca

Adding feature loadings for pca

Adding dimensional reduction information for umap



In [15]:
?Convert

Convert {SeuratDisk},R Documentation
source,Source dataset
dest,Name of destination dataset
assay,Converting from h5Seurat: name of assay to write out; converting to h5Seurat: name to store assay data as
overwrite,Overwrite existing dest
verbose,Show progress updates
...,Arguments passed to other methods


In [19]:
obj@assays$SCT@data

   [[ suppressing 32 column names ‘SI-TT-B10_AAGCATCAGGGAGATA_2’, ‘SI-TT-B2_ATTCGTTCACATGGTT_2’, ‘SI-TT-E1_TTGAACGGTTGTGCCG_2’ ... ]]

   [[ suppressing 32 column names ‘SI-TT-B10_AAGCATCAGGGAGATA_2’, ‘SI-TT-B2_ATTCGTTCACATGGTT_2’, ‘SI-TT-E1_TTGAACGGTTGTGCCG_2’ ... ]]

   [[ suppressing 32 column names ‘SI-TT-B10_AAGCATCAGGGAGATA_2’, ‘SI-TT-B2_ATTCGTTCACATGGTT_2’, ‘SI-TT-E1_TTGAACGGTTGTGCCG_2’ ... ]]



22567 x 20000 sparse Matrix of class "dgCMatrix"
                                                                                        
Hps1               0.6931472 .         .         .         .         0.6931472 0.6931472
AC096317.5         .         .         .         .         .         .         .        
Ccdc83             .         .         .         .         .         .         .        
Cnksr3             .         .         .         .         .         .         .        
Neurl1             .         1.3862944 .         .         1.0986123 .         0.6931472
Gsto1              .         .         .         .         .         0.6931472 0.6931472
Gsto2              .         0.6931472 .         0.6931472 .         .         .        
Art2b              .         .         .         .         .         .         .        
Cfap58             .         .         .         .         .         .         .        
Arid1b             1.3862944 1.3862944 1.0986123 1.6094379 0.